# AA228/CS238 Optional Final Project: Escape Roomba

This notebook tests the QMDP + Branch & Bound (sampling) implementation

In [1]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
#---

"/home/colasg/Documents/AA228FinalProject/Project.toml"

In [2]:
# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Statistics # to evaluate policy
using Printf
using JLD # to save alpha vectors

In [3]:
# useful solvers
using QMDP
using FIB

## Define the POMDP

### Create state space, action space, sensor and construct POMDP

The QMDP offline method compute 1 alpha vector $\alpha_a$ per action, with components $\alpha_a(s)$ for $s \in \mathcal{S}, a \in \mathcal{A}$

This methods only works with finite state and action spaces, we first define the discretization

Then we instantiate a Bump sensor. The Bumper indicates when contact has been made between any part of the Roomba and any wall.

Next, we instantiate the MDP, which defines the underlying simulation environment, assuming full observability. The MDP takes many arguments to specify details of the problem. One argument we must specify here is the ```config```. This argument, which can take values 1,2, or 3, specifies the room configuration, with each configuration corresponding to a different location for the goal and stairs.

Finally, we instantiate the POMDP. The POMDP takes as an argument the underlying MDP as well as the sensor, which it uses to define the observation model. 

In [4]:
# discrete state space
num_x_pts = 50
num_y_pts = 50
num_th_pts = 20
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts);

In [5]:
# discrete action space
vlist = [0, 5, 10]
omlist = [-1, 0, 1]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist));

In [6]:
sensor = Bumper()
config = 1 # 1,2, or 3
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace, aspace=aspace));

println("Number of discrete states:", n_states(m))
println("Number of discrete actions:", n_actions(m))

Number of discrete states:150000
Number of discrete actions:9


### Setting up a Particle Filter

Here, as the state space is high dimensional, we instantiate a particle filter.

First, we instantiate a resampler, which is responsible for updating the belief state given an observation. The first argument for both resamplers is the number of particles that represent the belief state. The lidar resampler takes a low-variance resampler as an additional argument, which is responsible for efficiently resampling a weighted set of particles. 

Next, we instantiate a ```SimpleParticleFilter```, which enables us to perform our belief updates.

Finally, we pass this particle filter into a custom struct called a ```RoombaParticleFilter```, which takes two additional arguments. These arguments specify the noise in the velocity and turn-rate, used when propegating particles according to the action taken. These can be tuned depending on the type of sensor used.

In [7]:
num_particles = 2000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

## Solve the POMDP

### Compute an upper bound on the Value function : QMDP

We use QMDP to compute one alpha vector $\alpha_a, a \in \mathcal{A}$ per discrete action, over the discrete state space

With the update rule :
$\alpha_a(s) = R(s,a) + \gamma \sum_{s'} T(s'|s, a) max_{a'} \alpha_{a'}(s')$

Here, we have a physical problem : we should have a deterministic $s' =$ POMDPs.transition$(m, s, a)$

This is not exactly the case in our setup as we have some noise on the action $a = (v, \omega)$

But in our computation of QMDP upper bound, we will assume the transition is purely deterministic

QMDP complexity: $\mathcal{O}((|\mathcal{S}| |\mathcal{A}|^2)^{n_{iter}})$

In [8]:
function QMDPSolver_new(m::RoombaModel, max_iterations::Int64=20, tolerance::Float64=1e-3)
    # mdp characteristics
    sspace = states(m)
    aspace = actions(m)
    # initialize the alpha vectors to 0s
    alphas = zeros(length(aspace), length(sspace))
    # number of iterations
    count = 1
    # norm of difference between current and previous value of alphas
    diff = Inf
    while (count <= max_iterations) & (diff > tolerance) 
        println("Iteration:", count, "; Norm difference:", diff)
        alphas_prev = copy(alphas)
        
        # iterate over the alpha vectors
        for (i, a) in enumerate(aspace)
            # iterate over the states
            for (j, s) in enumerate(sspace)
                # compute the next (deterministic) state
                sp = rand(transition(m, s, a))
                k = stateindex(m, sp)
                # update the alpha vectors (Value Iteration)
                alphas[i,j] = reward(m, s, a, sp) + discount(m) * maximum(alphas[:,k])
            end
        end
        count += 1
        diff = sum((alphas - alphas_prev).^2)
    end
    
    alphas_list = [alphas[i,:] for i in 1:length(aspace)]
    return alphas_list
    end;

In [9]:
QMDP_new_alphas = QMDPSolver_new(m);

Iteration:1; Norm difference:Inf
Iteration:2; Norm difference:6.997819976609832e7
Iteration:3; Norm difference:4.318514790034439e8
Iteration:4; Norm difference:4.0592233411308384e8
Iteration:5; Norm difference:2.0665909403661293e8
Iteration:6; Norm difference:8.59192110758929e7
Iteration:7; Norm difference:3.547057227238106e7
Iteration:8; Norm difference:1.5093154589533575e7
Iteration:9; Norm difference:6.259901479895057e6
Iteration:10; Norm difference:2.573626018240386e6
Iteration:11; Norm difference:1.0724354760654345e6
Iteration:12; Norm difference:436901.1131067977
Iteration:13; Norm difference:170631.84355938958
Iteration:14; Norm difference:61552.940716712204
Iteration:15; Norm difference:21978.080732163326
Iteration:16; Norm difference:8690.199328068618
Iteration:17; Norm difference:4223.4022323670615
Iteration:18; Norm difference:2669.532100778544
Iteration:19; Norm difference:2044.0958719064604
Iteration:20; Norm difference:1728.0367626223156


In [10]:
# initialize the solver
# key-word args are the maximum number of iterations the solver will run for, and the Bellman tolerance
solver = QMDPSolver(max_iterations=20, tolerance=1e-3) 

# solve for the QMDP policy
QMDP_policy = solve(solver, m);

In [11]:
# initialize the solver
# key-word args are the maximum number of iterations the solver will run for, and the Bellman tolerance
solver = FIBSolver() #max_iterations=20, tolerance=1e-3) 

# solve for the QMDP policy
FIB_policy = solve(solver, m);

MethodError: MethodError: no method matching reward(::RoombaPOMDP{Bumper,Bool}, ::RoombaState, ::RoombaAct)
Closest candidates are:
  reward(::Union{RoombaMDP, RoombaPOMDP}, ::AbstractArray{Float64,1}, ::AbstractArray{Float64,1}, !Matched::AbstractArray{Float64,1}) at /home/colasg/Documents/AA228FinalProject/src/roomba_env.jl:347
  reward(::Union{MDP, POMDP}, ::Any, ::Any, !Matched::Any) at /home/colasg/.julia/packages/POMDPs/JiYXY/src/pomdp.jl:107
  reward(!Matched::POMDPModelTools.FullyObservablePOMDP{S,A}, ::S, ::A) where {S, A} at /home/colasg/.julia/packages/POMDPModelTools/eHEjm/src/fully_observable_pomdp.jl:41
  ...

### Save the alpha vectors

In [13]:
JLD.save("QMDP_new_alphas.jld", "QMDP_new_alphas", QMDP_new_alphas)
JLD.save("QMDP_alphas.jld", "QMDP_alphas", QMDP_policy.alphas)
#save("FIB_alphas.jld", "FIB_alphas", FIB_policy.alphas)


### Load the alpha vectors

In [8]:
QMDP_new_alphas = load("QMDP_new_alphas.jld")["QMDP_new_alphas"]
QMDP_alphas = load("QMDP_alphas.jld")["QMDP_alphas"];
#load("FIB_alphas.jld")["FIB_alphas"]


### Define a policy : Branch and Bound

We define thr 'Branch and Bound' online policy that find the best action 'a' to take in state 's' by branching every possible series of action up to depth 'd'. The pruning process uses QMDP alpha vectors as an upper bound.

The worst state in which you can be is "falling in the stairs" which is a terminal state. We use the corresponding reward as a lower bound: $U_{lower} = R("fall", a) = -10$

First we create a struct that subtypes the Policy abstract type, defined in the package ```POMDPPolicies.jl```. Here, we can also define certain parameters, such as a variable defining the depth 'd'.

Next, we define a function that can take in our policy and the belief state and return the desired action. We do this by defining a new ```POMDPs.action``` function that will work with our policy. 

Complexity: $\mathcal{O}((|\mathcal{A}| n_{samples} + n_{particles})^{depth})$

In [9]:
# Define the policy to test
mutable struct ToEnd <: Policy
    d::Int64 # depth of forward search
end

function branch_bound(m::RoombaModel, belief_updater::RoombaParticleFilter, alphas::Array{Array{Float64,1},1}, b::ParticleCollection{RoombaState}, d::Int64)
    # hyperparameters
    n_samples = 5
    
    # mdp characteristics
    sspace = states(m)
    aspace = actions(m)    
    
    if d == 0
        U_lower = -10 # lower bound
        return (nothing, U_lower)
    end
    
    a_best, U_best = nothing, -Inf
    
    # compute the upper bounds for every actions from the alpha vectors
    U_upper = [mean([ alphas[i][stateindex(m, particle(b, j))] for j in 1:n_particles(b)]) for i in 1:length(aspace)]
    
    # argsort U_upper to 'prune' branches using this upper bound
    argsort_upper = sortperm(-U_upper)
    sorted_upper = U_upper[argsort_upper]
    
    # iterate over all the possible discrete actions
    for (i, a) in enumerate(aspace[argsort_upper])
        
        if sorted_upper[i] < U_best
            return a_best, U_best
        end
        
        U = 0
        
        # sample 'n_sample' observations
        for count=1: n_samples
            # sample a random particle from the belief
            n_part = n_particles(b)
            j = rand(1:n_part)
            s = particle(b, j)
            # sample a next state
            sp = generate_s(m, s, a, belief_updater.spf.rng)
            # sample an observation and a reward
            (o, r) = generate_or(m, s, a, sp, belief_updater.spf.rng)
            
            # update belief
            bp = update(belief_updater, b, a, o)

            _, Up = branch_bound(m, belief_updater, alphas, bp, d-1)
            # update the expected return
            U = U + 1/n_samples * (r + discount(m) * Up)
        end
        
        # force the Roomba to move
        if (U > U_best) & ((a.v > 0) | (a.omega > 0))
            a_best, U_best = a, U
        end
    end
    
    return (a_best, U_best)
end

# define a new function that takes in the policy struct and current belief,
# and returns the desired action
function POMDPs.action(p::ToEnd, b::ParticleCollection{RoombaState})
    a_best, U_best = branch_bound(m, belief_updater, QMDP_alphas, b, p.d)
    return a_best
end

### Simulation and rendering

Here, we will demonstrate how to seed the environment, run a simulation, and render the simulation. To render the simulation, we use the ```Gtk``` package. 

The simulation is carried out using the ```stepthrough``` function defined in the package ```POMDPSimulators.jl```. During a simulation, a window will open that renders the scene. It may be hidden behind other windows on your desktop.

In [13]:
# first seed the environment
Random.seed!(2)

# reset the policy
d = 1
p = ToEnd(d) # here, the argument sets the depth of the Branch & Bound search

# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, p, belief_updater, max_steps=100))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.01) # to slow down the simulation
end

InterruptException: InterruptException:

### Specifying initial states and beliefs
If, for debugging purposes, you would like to hard-code a starting location or initial belief state for the roomba, you can do so by taking the following steps.

First, we define the initial state using the following line of code:
```
is = RoombaState(x,y,th,0.)
```
Where ```x``` and ```y``` are the x,y coordinates of the starting location and ```th``` is the heading in radians. The last entry, ```0.```, respresents whether the state is terminal, and should remain unchanged.

If you would like to initialize the Roomba's belief as perfectly localized, you can do so with the following line of code:
```
b0 = Deterministic(is)
```
If you would like to initialize the standard unlocalized belief, use these lines:
```
dist = initialstate_distribution(m)
b0 = initialize_belief(belief_updater, dist)
```
Finally, we call the ```stepthrough``` function using the initial state and belief as follows:
```
stepthrough(m,planner,belief_updater,b0,is,max_steps=300)
```

### Evaluation 

Here, we demonstate a simple evaluation of the policy's performance for a few random seeds. This is meant to serve only as an example, and we encourage you to develop your own evaluation metrics.

We intialize the robot using five different random seeds, and simulate its performance for 100 time-steps. We then sum the rewards experienced during its interaction with the environment and track this total reward for the five trials.
Finally, we report the mean and standard error for the total reward. The standard error is the standard deviation of a sample set divided by the square root of the number of samples, and represents the uncertainty in the estimate of the mean value.

In [12]:
total_rewards = []

d = 1
for exp = 1:5    
    Random.seed!(exp)
    
    p = ToEnd(d)
    traj_rewards = sum([step.r for step in stepthrough(m,p,belief_updater, max_steps=100)])
    
    println("Experience: ", string(exp), " Reward: ", traj_rewards)

    push!(total_rewards, traj_rewards)
end

@printf("Mean Total Reward: %.3f, StdErr Total Reward: %.3f", mean(total_rewards), std(total_rewards)/sqrt(5))

Experience: 1 Reward: 2.499999999999999
Experience: 2 Reward: -14.0
Experience: 3 Reward: 3.3000000000000025
Experience: 4 Reward: 7.0
Experience: 5 Reward: 6.8
Mean Total Reward: 1.120, StdErr Total Reward: 3.887

In [14]:
total_rewards = []

d = 1
for exp = 6:10    
    Random.seed!(exp)
    
    p = ToEnd(d)
    traj_rewards = sum([step.r for step in stepthrough(m,p,belief_updater, max_steps=100)])
    
    println("Experience: ", string(exp), " Reward: ", traj_rewards)

    push!(total_rewards, traj_rewards)
end

@printf("Mean Total Reward: %.3f, StdErr Total Reward: %.3f", mean(total_rewards), std(total_rewards)/sqrt(5))

Experience: 6 Reward: 4.7
Experience: 7 Reward: -12.0
Experience: 8 Reward: -12.0
Experience: 9 Reward: -13.0
Experience: 10 Reward: -11.0
Mean Total Reward: -8.660, StdErr Total Reward: 3.355

In [15]:
total_rewards = []

d = 1
for exp = 11:15    
    Random.seed!(exp)
    
    p = ToEnd(d)
    traj_rewards = sum([step.r for step in stepthrough(m,p,belief_updater, max_steps=100)])
    
    println("Experience: ", string(exp), " Reward: ", traj_rewards)

    push!(total_rewards, traj_rewards)
end

@printf("Mean Total Reward: %.3f, StdErr Total Reward: %.3f", mean(total_rewards), std(total_rewards)/sqrt(5))

Experience: 11 Reward: -14.000000000000002
Experience: 12 Reward: -11.0
Experience: 13 Reward: 9.9
Experience: 14 Reward: -12.0
Experience: 15 Reward: -12.0
Mean Total Reward: -7.820, StdErr Total Reward: 4.457

In [ ]:
total_rewards = []

d = 1
for exp = 16:20    
    Random.seed!(exp)
    
    p = ToEnd(d)
    traj_rewards = sum([step.r for step in stepthrough(m,p,belief_updater, max_steps=100)])
    
    println("Experience: ", string(exp), " Reward: ", traj_rewards)

    push!(total_rewards, traj_rewards)
end

@printf("Mean Total Reward: %.3f, StdErr Total Reward: %.3f", mean(total_rewards), std(total_rewards)/sqrt(5))

Experience: 16 Reward: 7.199999999999999
Experience: 17 Reward: -13.0
Experience: 18 Reward: -16.0
Experience: 